In [4]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

data=pd.read_csv('sample_data/movie_data.csv')
data.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [5]:
data['review'][6]

'Nathan Detroit (Frank Sinatra) is the manager of the New York\'s longest- established floating craps game, and he needs $1000 to secure a new location. Confident of his odds, he bets the city\'s highest-roller, Sky Masterson (Marlon Brando), that he can\'t woo uptight missionary Sarah Brown (Jean Simmons). \'Guys and Dolls (1955)\' is such a great musical because it deftly blends the contrasting styles of film and stage. During a dazzling opening sequence, crowds of pedestrians move in rhythm, stopping and starting as though responding to backstage cues. Even the walking movements themselves are stylised and angular, halfway between a walk and a dance. Mankiewicz\'s New York City is a glittering flurry of art deco colour and movement, a fantasy world so completely removed from reality that even the business of underground gambling and criminal thuggery seems perfectly genial. <br /><br />As I write this review, I\'ve just received word that Jean Simmons has passed away, age 80. This, 

In [ ]:
count=CountVectorizer()

example=(['The sun is shining',
'The weather is sweet',
'The sun is shining, the weather is sweet, and one and one is two'])
bag=count.fit_transform(example)

In [ ]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [ ]:
np.set_printoptions(precision=2)
tfidf=TfidfTransformer(use_idf=True,norm ='l2',smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(ex)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [6]:
data.loc[0,'review'][-45:]

'ven.<br /><br />Title (Brazil): Not Available'

In [ ]:
def preprocess(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [ ]:
preprocess("</a>This :) is a test  :-)!")

'this is a test :) :)'

In [ ]:
data['review']=data['review'].apply(preprocess)

In [ ]:
porter_func=PorterStemmer()

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
def tokenizer_porter(text):
    return [porter_func.stem(word) for word in text.split()]

In [ ]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [ ]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SUBHAJIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words_list=stopwords.words('english')
[word for word in tokenizer_porter('a runner likes running and runs a lot')[-10:] if word not in stop_words_list]

['runner', 'like', 'run', 'run', 'lot']

In [ ]:
tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,tokenizer=tokenizer_porter,
                     use_idf=True,norm='l2',smooth_idf=True)


In [ ]:
y=data.sentiment.values
X=tfidf.fit_transform(data.review)

In [ ]:
X_train,X_test, y_train,y_test=train_test_split(X,y,random_state=1,
                                                test_size=0.5,shuffle=False)

In [ ]:
trained_classifier=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=1,n_jobs=-1,verbose=2,max_iter=200).fit(X_train,y_train)

saved_model=open('model_saved.sav','wb')
pickle.dump(trained_classifier,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [ ]:
filename='model_saved.sav'
saved_classifier=pickle.load(open(filename,'rb'))


In [ ]:
saved_classifier.score(X_test,y_test)

0.89608